In [136]:
import numpy as np # array operations
import cv2 #opencv library read img operations
import os # use directory & join path
from pathlib import Path # 去掉副檔名
import pandas as pd # 匯出 csv
import struct

# data visualisation and manipulation
import matplotlib.pyplot as plt #show img and table

In [137]:
getBin = lambda x: x > 0 and str(bin(x))[2:] or "-" + str(bin(x))[3:]
 
def floatToBinary64(value):
    val = struct.unpack('Q', struct.pack('d', value))[0]
    return getBin(val)
 
def binaryToFloat(value):
    hx = hex(int(value, 2))   
    return struct.unpack("d", struct.pack("q", int(hx, 16)))[0]

def dec_to_bin_csv():
    DATASRC = 'feature/dec/'
    # 抓目錄下所有圖檔檔名
    src_name = os.listdir(DATASRC)
    for i in range (len(src_name)):

        # 檔案完整路徑
        src_path = DATASRC + src_name[i]
        data = pd.read_csv(src_path,header=None)
        
        mean = [floatToBinary64(float(data[1][1])),floatToBinary64(float(data[2][1])),floatToBinary64(float(data[3][1]))]
        std = [floatToBinary64(float(data[1][2])),floatToBinary64(float(data[2][2])),floatToBinary64(float(data[3][2]))]
        weight = [floatToBinary64(float(data[1][3])),floatToBinary64(float(data[2][3])),floatToBinary64(float(data[3][3]))]
        
        MeanStd = [mean,std,weight]
        MeanStd_table = pd.DataFrame(MeanStd)
        col_name = ['red','green','blue']
        row_name = ['Mean','Standard Deviation','weight']
        MeanStd_table.columns = col_name
        MeanStd_table.index = row_name
        # print(MeanStd_table)

        # 匯出 answer table 成 csv 檔
        src_name_temp= Path(src_name[i]).stem 
        src_name_tempp = src_name_temp.rstrip('dec')
        MeanStd_table.to_csv('output/bin/'+ src_name_tempp + 'bin.csv')

In [138]:
def get_mean_and_std(typeStr, img_name, x, x_weight):
    x_mean, x_std = cv2.meanStdDev(x)
    x_mean = np.hstack(np.around(x_mean,2))
    x_std = np.hstack(np.around(x_std,2))
    
    # 印出平均與標準差
    if(typeStr == 's'):
        print("\n來源圖檔名: ", img_name)
    elif(typeStr == 't'):
        print("\n目標圖檔名: ", img_name)
    
    MeanStd = [x_mean,x_std,x_weight]
    MeanStd_table = pd.DataFrame(MeanStd)
    col_name = ['red','green','blue']
    row_name = ['Mean','Standard Deviation','weight']
    MeanStd_table.columns = col_name
    MeanStd_table.index = row_name
    print(MeanStd_table)

    # 匯出 answer table 成 csv 檔
    MeanStd_table.to_csv('feature/dec/'+ img_name + '_dec.csv')
    # 轉成 IEEE bin
    dec_to_bin_csv()
    
    return x_mean, x_std

In [139]:
def color_transfer(src_lab, tar_lab, src_name, tar_name, coltra_name, coltra_num, weight_r, weight_g, weight_b):
    
    x_weight=[weight_r, weight_g, weight_b]
    s_mean, s_std = get_mean_and_std('s', src_name, src_lab, x_weight)
    t_mean, t_std = get_mean_and_std('t', tar_name, tar_lab, x_weight)

    # height, width, channel = src_rgb.shape
    
    coltra_lab = src_lab

    for k in range(0,3):
        x = src_lab[:,:,k]
        x = ((x-s_mean[k])*(((x_weight[k]*t_std[k])+(1-x_weight[k])*s_std[k])/s_std[k]))+ x_weight[k]*t_mean[k]+(1-x_weight[k])*s_mean[k]

        # # round or +0.5
        # x = np.round(x)
        x = np.ceil(x)

        # boundary check 超過邊界拉回在邊界上
        x = np.clip(x, 0, 255)
        coltra_lab[:,:,k] = x
    
    # 存顏色轉換結果圖
    str_weight = '_' + str(weight_r) + '_' + str(weight_g) + '_' + str(weight_b)
    # coltra_lab = np.floor(coltra_lab * 255 + 0.5).astype(np.float32)
    # coltra_lab = coltra_lab.astype(np.float32)
    coltra_bgr = cv2.cvtColor(coltra_lab,cv2.COLOR_LAB2BGR)
    print(coltra_bgr)
    cv2.imwrite('result/'+ coltra_name + str_weight + '.png', coltra_bgr)
    

In [140]:
if __name__ == '__main__':

    weight_r, weight_g, weight_b = input("Enter 3 channel weight_red , weight_green, weight_blue(0.0 ≤ w ≤ 1.0):").split()
    
    weight_r = float(weight_r)
    weight_g = float(weight_g)
    weight_b = float(weight_b)
    
    # print("weight is: " + weight_r, weight_g, weight_b)
    # weight_r, weight_g, weight_b = 0.49,0.54,0.62

    # 印出所有圖片
    DATASRC = 'source/'
    DATATRG = 'target/'

    # 抓目錄下所有圖檔檔名
    src_name = os.listdir(DATASRC)
    tar_name = os.listdir(DATATRG)

    # for i in range (len(src_name)):
    for i in range (1):

            
        # 圖片完整路徑
        src_path = DATASRC + src_name[i]
        tar_path = DATATRG + tar_name[i]
        
        # convert img to array 以彩色格式讀取(三維)
        src_bgr = cv2.imread(src_path ,cv2.IMREAD_COLOR).astype(np.uint8)
        tar_bgr = cv2.imread(tar_path ,cv2.IMREAD_COLOR).astype(np.uint8)

        # 原為 BGR 轉為 Lab
        src_lab = cv2.cvtColor(src_bgr,cv2.COLOR_BGR2LAB)
        tar_lab = cv2.cvtColor(tar_bgr,cv2.COLOR_BGR2LAB)

        # normalize
        # src_lab = (src_lab / 255.0).astype(np.float32)  
        # tar_lab = (tar_lab / 255.
        # 0).astype(np.float32)  

        # 去掉副檔名
        src_name_temp= Path(src_path).stem 
        tar_name_temp= Path(tar_path).stem 
        tar_name_tempp = tar_name_temp.lstrip('0'+str(i+1))
        coltra_name = src_name_temp + tar_name_tempp
        
        # 做色彩轉換
        color_transfer(src_lab, tar_lab, src_name_temp, tar_name_temp, coltra_name, i+1, weight_r, weight_g, weight_b)



來源圖檔名:  01_kodim17
                      red   green    blue
Mean                82.07  129.16  133.52
Standard Deviation  53.09    3.34    6.74
weight               1.00    1.00    1.00

目標圖檔名:  01_kodim23
                       red   green    blue
Mean                120.17  128.55  148.69
Standard Deviation   47.27   21.61   19.69
weight                1.00    1.00    1.00
[[[126 137 127]
  [126 137 127]
  [126 137 127]
  ...
  [126 137 127]
  [126 137 127]
  [126 137 127]]

 [[126 137 127]
  [126 137 127]
  [126 137 127]
  ...
  [126 137 127]
  [126 137 127]
  [126 137 127]]

 [[126 137 127]
  [126 137 127]
  [127 138 128]
  ...
  [133 141 145]
  [126 137 127]
  [126 137 127]]

 ...

 [[126 137 127]
  [126 137 127]
  [126 137 127]
  ...
  [126 137 127]
  [126 137 127]
  [126 137 127]]

 [[126 137 127]
  [126 137 127]
  [126 137 127]
  ...
  [126 137 127]
  [126 137 127]
  [126 137 127]]

 [[126 137 127]
  [126 137 127]
  [126 137 127]
  ...
  [126 137 127]
  [126 137 127]
  [126 1